## More advanced matching and manipulation

Now for two advanced ways to use regular expressions along with stringr: selecting parts of a match (a.k.a capturing) and referring back to parts of a match (a.k.a back-referencing). You'll also learn to deal with and strings or patterns that contain Unicode characters (e.g. é).

### Capturing parts of a pattern
In rebus, to denote a part of a regular expression you want to capture, you surround it with the function capture(). For example, a simple pattern to match an email address might be,

email <- one_or_more(WRD) %R% 
  "@" %R% one_or_more(WRD) %R% 
  DOT %R% one_or_more(WRD)
str_view("(wolverine@xmen.com)", pattern = email)  
If you want to capture the part before the @, you simply wrap that part of the regular expression in capture():

email <- capture(one_or_more(WRD)) %R% 
  "@" %R% one_or_more(WRD) %R% 
  DOT %R% one_or_more(WRD)
str_view("(wolverine@xmen.com)", pattern = email)  
The part of the string that matches hasn't changed, but if we pull out the match with str_match() we get access to the captured piece:

str_match("(wolverine@xmen.com)", pattern =  email)  
You'll explore this behavior with some more super hero email addresses.

In [3]:
library(rebus)
library(stringr)
hero_contacts = c("(wolverine@xmen.com)","wonderwoman@justiceleague.org", "thor@avengers.com")            

email <- one_or_more(WRD) %R% 
  "@" %R% one_or_more(WRD) %R% 
  DOT %R% one_or_more(WRD)
str_view(hero_contacts, pattern = email)


Attaching package: 'stringr'

The following object is masked from 'package:rebus':

    regex



HTML widgets cannot be represented in plain text (need html)

In [6]:
# Add some more capture() calls to the email pattern, to capture the part after the @ but before the .; 
# and the part after the ..

# Capture parts between @ and . and after .
email <- capture(one_or_more(WRD)) %R% 
  "@" %R% capture(one_or_more(WRD)) %R% 
  DOT %R% capture(one_or_more(WRD))

# Check that the whole email address is matched by using str_view() on hero_contacts with the email pattern.

# Check match hasn't changed
str_view(hero_contacts, pattern = email)

HTML widgets cannot be represented in plain text (need html)

In [7]:
# Use str_match() to pull out the match and capture the email pattern in hero_contacts, 
# and print it to confirm all parts are captured.

# Pull out match and captures
email_parts <- str_match(hero_contacts, pattern = email)
email_parts

# Assign the third column of email_parts to host and print it.# Save host
host <- email_parts[,3]
host


wolverine@xmen.com,wolverine,xmen,com
wonderwoman@justiceleague.org,wonderwoman,justiceleague,org
thor@avengers.com,thor,avengers,com


[1] "xmen"          "justiceleague" "avengers"

### Pulling out parts of a phone number
You can now go back to the phone number example from the previous chapter. You developed a pattern to extract the parts of string that looked like phone numbers, and now you have the skills to pull out the pieces of the number. Let's see if you can put your skills together to output the first phone number in each string in a common format.

In [9]:
contact = c("Call me at 555-555-0191", "123 Main St", "(555) 555 0191", "Phone: 555.555.0191 Mobile: 555.555.0192")

# Use these components
three_digits <- DGT %R% DGT %R% DGT
four_digits <- three_digits %R% DGT
separator <- char_class("-.() ")

# phone_pattern is the pattern you developed in the last chapter. Edit it to capture() each sequence of numbers.
phone_pattern <- optional(OPEN_PAREN) %R%
  capture(three_digits) %R%
  zero_or_more(separator) %R%
  capture(three_digits) %R% 
  zero_or_more(separator) %R%
  capture(four_digits)

# Pull out the parts with str_match()
phone_numbers <- str_match(contact, pattern = phone_pattern)

# Put them back together into the format (XXX) XXX-XXXX.
str_c("(",
  phone_numbers[, 2],
  ") ",
  phone_numbers[,3],
  "-",
  phone_numbers[,4])


[1] "(555) 555-0191" NA               "(555) 555-0191" "(555) 555-0191"

In [10]:
# Extracting age and gender again
# You can now also take another look at your pattern for pulling out age and gender from the injury narratives. 
# In the last chapter you took two steps to extract the pattern and then pull it apart. Take a go at doing it in one step!

# load narrative rds file
narr <- readRDS(file = "narratives.rds")

# Add capture() to get age, unit and sex
pattern <- capture(optional(DGT) %R% DGT) %R% 
  optional(SPC) %R% capture(or("YO", "YR", "MO")) %R%
  optional(SPC) %R% capture(or("M", "F"))

# Pull out from narratives
str_match(narr, pattern = pattern)

19YOM,19,YO,M
31 YOF,31,YO,F
82 YOM,82,YO,M
33 YOF,33,YO,F
10YOM,10,YO,M
53 YO F,53,YO,F
13 MOF,13,MO,F
14YR M,14,YR,M
55YOM,55,YO,M
5 YOM,5,YO,M


In [12]:
# That's pretty good, but it might be nice just to capture the first letter of the units, e.g. the Y or M. 
# Alter your pattern to capture a Y or M, and then optional()ly match O or R.

# Edit to capture just Y and M in units
pattern2 <- capture(optional(DGT) %R% DGT) %R%  
  optional(SPC) %R% capture(or("Y", "M")) %R% optional(or("O","R")) %R%
  optional(SPC) %R% capture(or("M", "F"))

# Check pattern
str_view(narr, pattern2)

# Pull out pieces
str_match(narr, pattern2)

HTML widgets cannot be represented in plain text (need html)

19YOM,19,Y,M
31 YOF,31,Y,F
82 YOM,82,Y,M
33 YOF,33,Y,F
10YOM,10,Y,M
53 YO F,53,Y,F
13 MOF,13,M,F
14YR M,14,Y,M
55YOM,55,Y,M
5 YOM,5,Y,M


### Using backreferences in patterns
Backreferences can be useful in matching because they allow you to find repeated patterns or words. Using a backreference requires two things: you need to capture() the part of the pattern you want to reference, and then you refer to it with REF1.

Take a look at this pattern: capture(LOWER) %R% REF1. It matches and captures any lower case character, then is followed by the captured character: it detects repeated characters regardless of what character is repeated. To see it in action try this:

str_view(c("hello", "sweet", "kitten"), 
  pattern = capture(LOWER) %R% REF1)
If you capture more than one thing you can refer to them with REF2, REF3 etc. up to REF9, counting the captures from the left of the pattern.


In [24]:
library(babynames)
library(dplyr)
library(stringr)

# Extracting vectors for boys' and girls' names
babynames_2014 <- filter(babynames, year == 2014)
boy_names <- filter(babynames_2014, sex == "M")$name
girl_names <- filter(babynames_2014, sex == "F")$name

# convert all names to lower case

boy_names = str_to_lower(boy_names)

# In each case, assign the pattern argument, then view the matches by running the str_view() code.

# See all the boy_names with a letter repeated three times, by extending the pattern in the text above with another REF1. 
# Assign the pattern to repeated_three_times.

# Names with three repeated letters
repeated_three_times <- capture(LOWER) %R% REF1 %R% REF1

# Test it
str_view(boy_names, pattern = repeated_three_times, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [26]:
# See all the boy_names with a pair of letters repeated twice, e.g. abab, by capturing two lower case characters, 
# then referring to the capture with REF1. Assign the pattern to pair_of_repeated.

# Names with a pair of repeated letters
pair_of_repeated <- capture(LOWER %R% LOWER) %R% REF1

# Test it
str_view(boy_names, pattern = pair_of_repeated, match = TRUE)


HTML widgets cannot be represented in plain text (need html)

In [27]:
# See all the boy_names with a pair of letter followed by their reverse, e.g. abba, by capturing two lower case characters 
# separately and combining with REF2 and REF1. Assign the pattern to pair_that_reverses.

# Names with a pair that reverses
pair_that_reverses <- capture(LOWER) %R% capture(LOWER) %R% REF2 %R% REF1

# Test it
str_view(boy_names, pattern = pair_that_reverses, match = TRUE)

HTML widgets cannot be represented in plain text (need html)

In [28]:
# See all the boy_names that are a four letter palindrome (a name that reads the same forwards and backwards) by 
# wrappping the previous pattern in exactly(). Assign the pattern to four_letter_palindrome.

# Four letter palindrome names
four_letter_palindrome <- exactly(capture(LOWER) %R% capture(LOWER) %R% REF2 %R% REF1)

# Test it
str_view(boy_names, pattern = four_letter_palindrome, match = TRUE)


HTML widgets cannot be represented in plain text (need html)

### Replacing with regular expressions
Now, you've mastered matching with backreferences, you'll build up to replacing with backreferences, but first let's review str_replace() now that you've got regular expressions under your belt.

Remember str_replace() takes three arguments, string a vector of strings to do the replacements in, pattern that identifies the parts of strings to replace and replacement the thing to use as a replacement.

replacement can be a vector, the same length as string, each element specifies the replacement to be used in each string. Let's practice by anonymizing some of the contact objects you've seen so far.

In [30]:
# Replace digits with "X"
str_replace(contact, DGT, replacement = "X")

# Replace all digits with "X"
str_replace_all(contact, DGT, replacement = "X")

# Replace all digits with different symbol
str_replace_all(contact, DGT, replacement = c("X", ".","*", "_"))

[1] "Call me at X55-555-0191"                 
[2] "X23 Main St"                             
[3] "(X55) 555 0191"                          
[4] "Phone: X55.555.0191 Mobile: 555.555.0192"

[1] "Call me at XXX-XXX-XXXX"                 
[2] "XXX Main St"                             
[3] "(XXX) XXX XXXX"                          
[4] "Phone: XXX.XXX.XXXX Mobile: XXX.XXX.XXXX"

[1] "Call me at XXX-XXX-XXXX"                 
[2] "... Main St"                             
[3] "(***) *** ****"                          
[4] "Phone: ___.___.____ Mobile: ___.___.____"

### Replacing with backreferences
The replacement argument to str_replace() can also include backreferences. This works just like specifying patterns with backreferences, except the capture happens in the pattern argument, and the backreference is used in the replacement argument.

x <- c("hello", "sweet", "kitten")
str_replace(x, capture(ANY_CHAR), str_c(REF1, REF1))
capture(ANY_CHAR) will match the first character no matter what it is. Then the replacement str_c(REF1, REF1) combines the captured character with itself, in effect doubling the first letter of each string.

You are going to use this to create some alternative, more lively accident narratives.

The strategy you'll take is to match words ending in "ING" then replace them with an adverb followed by the original word.

In [43]:
# load narrative rds file
adverbs <- readRDS(file = "adverbs.rds")

# Build pattern to match words ending in "ING"
pattern <- wrd(1,Inf) %R% "ING"
str_view(narr, pattern)

# Test replacement and a and a replacement str_c("CARELESSLY", REF1, sep = " ").
str_replace(narr, capture(pattern), replacement = str_c("CARELESSLY", REF1, sep = " "))

# One adverb per narrative: Build a vector with one adverb for each narrative by sampling 10 elements from adverbs.
adverbs_10 <- sample(adverbs, 10)

# Replace "***ing" with "adverb ***ly"
str_replace(narr, capture(pattern), replacement = str_c(adverbs_10, REF1, sep = " "))

HTML widgets cannot be represented in plain text (need html)

[1] "19YOM-SHOULDER STRAIN-WAS TACKLED WHILE CARELESSLY PLAYING FOOTBALL W/ FRIENDS "                      
 [2] "31 YOF FELL FROM TOILET HITITNG HEAD CARELESSLY SUSTAINING A CHI "                                    
 [3] "ANKLE STR. 82 YOM STRAINED ANKLE CARELESSLY GETTING OUT OF BED "                                      
 [4] "TRIPPED OVER CAT AND LANDED ON HARDWOOD FLOOR. LACERATION ELBOW, LEFT. 33 YOF*"                       
 [5] "10YOM CUT THUMB ON METAL TRASH CAN DX AVULSION OF SKIN OF THUMB "                                     
 [6] "53 YO F TRIPPED ON CARPET AT HOME. DX HIP CONTUSION "                                                 
 [7] "13 MOF CARELESSLY TRYING TO STAND UP HOLDING ONTO BED FELL AND HIT FOREHEAD ON RADIATOR DX LACERATION"
 [8] "14YR M CARELESSLY PLAYING FOOTBALL; DX KNEE SPRAIN "                                                  
 [9] "55YOM RIDER OF A BICYCLE AND FELL OFF SUSTAINED A CONTUSION TO KNEE "                                 
[10] "5 YOM CARELESSLY ROLLING ON FLOOR DOING A SOMERSAULT AND SUSTAINED A CERVICAL STRA IN"

[1] "19YOM-SHOULDER STRAIN-WAS TACKLED WHILE COOLLY PLAYING FOOTBALL W/ FRIENDS "                       
 [2] "31 YOF FELL FROM TOILET HITITNG HEAD REGULARLY SUSTAINING A CHI "                                  
 [3] "ANKLE STR. 82 YOM STRAINED ANKLE STERNLY GETTING OUT OF BED "                                      
 [4] "TRIPPED OVER CAT AND LANDED ON HARDWOOD FLOOR. LACERATION ELBOW, LEFT. 33 YOF*"                    
 [5] "10YOM CUT THUMB ON METAL TRASH CAN DX AVULSION OF SKIN OF THUMB "                                  
 [6] "53 YO F TRIPPED ON CARPET AT HOME. DX HIP CONTUSION "                                              
 [7] "13 MOF SHARPLY TRYING TO STAND UP HOLDING ONTO BED FELL AND HIT FOREHEAD ON RADIATOR DX LACERATION"
 [8] "14YR M SEPARATELY PLAYING FOOTBALL; DX KNEE SPRAIN "                                               
 [9] "55YOM RIDER OF A BICYCLE AND FELL OFF SUSTAINED A CONTUSION TO KNEE "                              
[10] "5 YOM FURIOUSLY ROLLING ON FLOOR DOING A SOMERSAULT AND SUSTAINED A CERVICAL STRA IN"

[1] "ABNORMALLY"     "ABSENTMINDEDLY" "ACCIDENTALLY"   "ACIDLY"        
 [5] "ACTUALLY"       "ADVENTUROUSLY"  "AFTERWARDS"     "ALMOST"        
 [9] "ALWAYS"         "ANGRILY"

### Matching a specific code point or code groups
Things can get tricky when some characters can be specified two ways, for example è, an e with a grave accent, can be specified either with the single code point \u00e8 or the combination of a \u0065 and a combining grave accent \u0300. They look the same:

x <- c("\u00e8", "\u0065\u0300")
writeLines(x)
But, specifying the single code point only matches that version:

str_view(x, "\u00e8")
The stringi package that stringr is built on contains functions for converting between the two forms. stri_trans_nfc() composes characters with combining accents into a single character. stri_trans_nfd() decomposes character with accents into separate letter and accent characters. You can see how the characters differ by looking at the hexadecimal codes.

as.hexmode(utf8ToInt(stri_trans_nfd("\u00e8")))
as.hexmode(utf8ToInt(stri_trans_nfc("\u0065\u0300")))
In Unicode, an accent is known as a diacritic Unicode Property, and you can match it using the rebus value UP_DIACRITIC.

Vietnamese makes heavy use of diacritics to denote the tones in its words. In this exercise, you'll manipulate the diacritics in the names of Vietnamese rulers.

In [47]:
install.packages("stringi")
library(stringi)

# Names of rulers from the 18th Century Vietnamese Tây Sơn dynasty are shown in the script.

# tay_son_builtin has the accents built into each letter. Run the code that defines and prints this variable.

# Names with builtin accents
(tay_son_builtin <- c(
  "Nguy\u1ec5n Nh\u1ea1c", 
  "Nguy\u1ec5n Hu\u1ec7",
  "Nguy\u1ec5n Quang To\u1ea3n"
))

# Call stri_trans_nfd() to decompose the letters with accents into separate letter and accent characters,
tay_son_separate <- stri_trans_nfd(tay_son_builtin)

# Verify that the string prints the same
tay_son_separate

# View all the accents by calling str_view_all() and matching UP_DIACRITIC. The match is shown after the letter 
# that the diacritic belongs to.

# Match all accents
str_view_all(tay_son_separate, UP_DIACRITIC)

package 'stringi' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'stringi'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Migue\Anaconda3\envs\withR\Lib\R\library\00LOCK\stringi\libs\icudt61l.dat to C:\Users\Migue\Anaconda3\envs\withR\Lib\R\library\stringi\libs\icudt61l.dat: Invalid argument"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Migue\Anaconda3\envs\withR\Lib\R\library\00LOCK\stringi\libs\x64\stringi.dll to C:\Users\Migue\Anaconda3\envs\withR\Lib\R\library\stringi\libs\x64\stringi.dll: Permission denied"Warning message:
"restored 'stringi'"


The downloaded binary packages are in
	C:\Users\Migue\AppData\Local\Temp\RtmpsVCGta\downloaded_packages


[1] "Nguy<U+1EC5>n Nh<U+1EA1>c" "Nguy<U+1EC5>n Hu<U+1EC7>" "Nguy<U+1EC5>n Quang To<U+1EA3>n"

[1] "Nguye^~n Nha<U+0323>c" "Nguye^~n Hue<U+0323>^" "Nguye^~n Quang Toa<U+0309>n"

HTML widgets cannot be represented in plain text (need html)

### Matching a single grapheme
A related problem is matching a single character. You've used ANY_CHAR to do this up until now, but it will only match a character represented by a single code point. Take these three names:

x <- c("Adele", "Ad\u00e8le", "Ad\u0065\u0300le")
writeLines(x)
They look the similar, but this regular expression only matches two of them:

str_view(x, "Ad" %R% ANY_CHAR %R% "le")
because in the third name è is represented by two code points. The Unicode standard has a concept of a grapheme that represents a display character, but may be composed of many code points. To match any grapheme you can use GRAPHEME.

str_view(x, "Ad" %R% GRAPHEME %R% "le")
Names of rulers from the Vietnamese Tây Sơn dynasty, with diacritics given as separate graphemes, is pre-defined as tay_son_separate.

In [48]:
# tay_son_separate has been pre-defined
tay_son_separate

# View all the characters in tay_son_separate
str_view_all(tay_son_separate, pattern = ANY_CHAR)

[1] "Nguye^~n Nha<U+0323>c" "Nguye^~n Hue<U+0323>^" "Nguye^~n Quang Toa<U+0309>n"

HTML widgets cannot be represented in plain text (need html)

In [50]:
# View all the graphemes in tay_son_separate
str_view_all(tay_son_separate, GRAPHEME)

HTML widgets cannot be represented in plain text (need html)

In [51]:
# Use stri_trans_nfc() to combine the diacritics with their associated characters, storing the result as tay_son_builtin.
tay_son_builtin <-  stri_trans_nfc(tay_son_separate)
tay_son_builtin

# View all the graphemes in tay_son_builtin
str_view_all(tay_son_builtin, GRAPHEME)

[1] "Nguy<U+1EC5>n Nh<U+1EA1>c" "Nguy<U+1EC5>n Hu<U+1EC7>" "Nguy<U+1EC5>n Quang To<U+1EA3>n"

HTML widgets cannot be represented in plain text (need html)